In [1]:
#Cellid

#read data

data=read.table("data/macparland/GSE115469_Data.csv",header=TRUE,row.names=1,sep=",")

data=t(data)
#annotation=BaronMetaData

In [2]:
annotation=read.table("data/macparland/GSE115469_CellClusterType.txt",header=TRUE,sep="\t",col.names=c("CellName","Sample","Cell#","Cluster#","cell.type"))
annotation$celltype=annotation$cell.type
head(annotation)

Warning message in read.table("data/macparland/GSE115469_CellClusterType.txt", header = TRUE, :
“header and 'col.names' are of different lengths”


,CellName,Sample,Cell.,Cluster.,cell.type,celltype
,<chr>,<chr>,<chr>,<int>,<chr>,<chr>
1,P1TLH_AAACCTGAGCAGCCTC_1,P1TLH,AAACCTGAGCAGCCTC,12,Central_venous_LSECs,Central_venous_LSECs
2,P1TLH_AAACCTGTCCTCATTA_1,P1TLH,AAACCTGTCCTCATTA,17,Cholangiocytes,Cholangiocytes
3,P1TLH_AAACCTGTCTAAGCCA_1,P1TLH,AAACCTGTCTAAGCCA,12,Central_venous_LSECs,Central_venous_LSECs
4,P1TLH_AAACGGGAGTAGGCCA_1,P1TLH,AAACGGGAGTAGGCCA,10,Non-inflammatory_Macrophage,Non-inflammatory_Macrophage
5,P1TLH_AAACGGGGTTCGGGCT_1,P1TLH,AAACGGGGTTCGGGCT,2,alpha-beta_T_Cells,alpha-beta_T_Cells
6,P1TLH_AAAGCAACAGTAAGAT_1,P1TLH,AAAGCAACAGTAAGAT,2,alpha-beta_T_Cells,alpha-beta_T_Cells


In [4]:
extract_prediction_label = function(x){
    y=c()
    for (i in x){
        y=c(y,i)
    }
    y
    return(y)
}



calculate_test_accuracy <- function(true_labels, predicted_labels) {
  if (length(true_labels) != length(predicted_labels)) {
    stop("Input vectors must have the same length.")
  }
  
  correct_predictions <- sum(true_labels == predicted_labels)
  total_samples <- length(true_labels)
  
  accuracy <- correct_predictions / total_samples
  return(accuracy)
}


calculate_f1_score_multiclass <- function(true_labels, predicted_labels) {
  if (length(true_labels) != length(predicted_labels)) {
    stop("Input vectors must have the same length.")
  }
  
  # Get unique class labels
  classes <- unique(c(true_labels, predicted_labels))
  
  # Initialize variables to store per-class statistics
  precision <- numeric(length(classes))
  recall <- numeric(length(classes))
  f1_score <- numeric(length(classes))
  
  # Calculate precision, recall, and F1 score for each class
  for (i in 1:length(classes)) {
    class_label <- classes[i]
    true_positive <- sum(predicted_labels == class_label & true_labels == class_label)
    false_positive <- sum(predicted_labels == class_label & true_labels != class_label)
    false_negative <- sum(predicted_labels != class_label & true_labels == class_label)
    
    precision[i] <- true_positive / (true_positive + false_positive)
    recall[i] <- true_positive / (true_positive + false_negative)
    
    f1_score[i] <- 2 * (precision[i] * recall[i]) / (precision[i] + recall[i])
  }
  
  # Calculate macro-averaged F1 score
  macro_f1_score <- mean(f1_score, na.rm = TRUE)
  
  return(macro_f1_score)
}




In [5]:
library(Seurat)
library(CelliD)
library(tidyverse) # general purpose library for data handling
library(ggpubr) #library for plotting

set.seed(124)


num_folds <- 5

# Generate 5-fold cross-validation indices
fold_indices <- split(sample(nrow(data)), 1:num_folds)

# Perform 5-fold cross-validation
for (i in 1:num_folds) {
  # Get indices for the current fold
  test_indices <- fold_indices[[i]]
  train_indices <- unlist(fold_indices[-i])
  
  # Subset data and annotation based on indices
  data_train <- data[train_indices, ]
  anno_train <- annotation[train_indices, ]
  
  data_test <- data[test_indices, ]
  anno_test <- annotation[test_indices, ]
  
  ref = CreateSeuratObject(counts = t(data_train))
  ref <- NormalizeData(ref)
  ref <- ScaleData(ref, verbose = FALSE)
  ref =  FindVariableFeatures(ref, selection.method = "vst", nfeatures = 2000, verbose = FALSE)
  ref <- RunPCA(ref, npcs = 30, verbose = FALSE)
  ref <- RunUMAP(ref, reduction = "pca", dims = 1:30, verbose = FALSE)
  
  query = CreateSeuratObject(counts = t(data_test))
  query <- NormalizeData(query)
  query <- ScaleData(query, verbose = FALSE)
  query =  FindVariableFeatures(query, selection.method = "vst", nfeatures = 2000, verbose = FALSE)
  query <- RunPCA(query, npcs = 30, verbose = FALSE)
  query <- RunUMAP(query, reduction = "pca", dims = 1:30, verbose = FALSE)
  query <- RunMCA(query, nmcs = 50)

  #cell match
  ref=RunMCA(ref)
  # Extracting per-cell gene signatures from the Baron dataset with CelliD(c)
  ref_cell_gs <- GetCellGeneSet(ref, dims = 1:50, n.features = 200)

  ref$celltype=anno_train$celltype


  # Extracting per-group gene signatures from the Baron dataset with CelliD(g)
  ref_group_gs <- GetGroupGeneSet(ref, dims = 1:50, n.features = 200, group.by = "celltype")
  
  
  #query <- FindVariableFeatures(query)
  #query <- ScaleData(query)
  #query <- RunMCA(query, nmcs = 50)

  #query <- RunPCA(query)
  #query <- RunUMAP(query, dims = 1:30)
  #query <- RunTSNE(query, dims = 1:30)
  


  HGT_ref_cell_gs <- RunCellHGT(query, pathways = ref_cell_gs, dims = 1:50)

  ref_cell_gs_match <- rownames(HGT_ref_cell_gs)[apply(HGT_ref_cell_gs, 2, which.max)]
  ref_cell_gs_prediction <- ref$celltype[ref_cell_gs_match]


  predictions=extract_prediction_label(ref_cell_gs_prediction)
  
  acc=calculate_test_accuracy(anno_test$celltype,predictions)


  f1_score = calculate_f1_score_multiclass(anno_test$celltype,predictions)


  cat("Fold", i, ": Train samples =", nrow(data_train), ", Test samples =", nrow(data_test), "Accuracy:", acc,"f1 score:",f1_score, "\n")

  
  # Perform training and testing using data_train, anno_train, data_test, and anno_test
}


Attaching SeuratObject

Loading required package: SingleCellExperiment

Loading required package: SummarizedExperiment

Loading required package: MatrixGenerics

Loading required package: matrixStats


Attaching package: ‘MatrixGenerics’


The following objects are masked from ‘package:matrixStats’:

    colAlls, colAnyNAs, colAnys, colAvgsPerRowSet, colCollapse,
    colCounts, colCummaxs, colCummins, colCumprods, colCumsums,
    colDiffs, colIQRDiffs, colIQRs, colLogSumExps, colMadDiffs,
    colMads, colMaxs, colMeans2, colMedians, colMins, colOrderStats,
    colProds, colQuantiles, colRanges, colRanks, colSdDiffs, colSds,
    colSums2, colTabulates, colVarDiffs, colVars, colWeightedMads,
    colWeightedMeans, colWeightedMedians, colWeightedSds,
    colWeightedVars, rowAlls, rowAnyNAs, rowAnys, rowAvgsPerColSet,
    rowCollapse, rowCounts, rowCummaxs, rowCummins, rowCumprods,
    rowCumsums, rowDiffs, rowIQRDiffs, rowIQRs, rowLogSumExps,
    rowMadDiffs, rowMads, rowMaxs, rowMeans2, r

0.998 sec elapsed


Computing SVD



10.283 sec elapsed


Computing Coordinates



...

29.92 sec elapsed


Computing Coordinates



1.918 sec elapsed



calculating distance



creating ranking



creating geneset



creating ranking



calculating distance


ranking genes

6756 pathways kept for hypergeometric test out of 6756, 0 filtered as less than 10 features was present in the data


calculating features overlap


performing hypergeometric test




Fold 5 : Train samples = 6756 , Test samples = 1688 Accuracy: 0.8299763 f1 score: 0.8615811 
